In [1]:
import cv2 as cv
import numpy as np
import random
import os
import math
import xml.etree.ElementTree as ET

In [2]:
def click_corners(img, objpoints, imgpoints, objp, window, n_rows=8, n_cols=6):
    """
    get 4 corners from user clicks (in order) and use linear interpolation to get the
    other inner points.
    """

    order = ["TL", "BL", "TR", "BR"]
    box_corners = []

    # function to handle clicks on the image
    def click_event(event, x, y, flags, param):

        box_corners, img, window = param

        # check for left mouse clicks
        if event == cv.EVENT_LBUTTONDOWN and len(box_corners) < 4:

            box_corners.append((x, y))

            # display clicked points
            # on the image window
            font = cv.FONT_HERSHEY_SIMPLEX
            cv.putText(
                img, order[len(box_corners) - 1], (x, y), font, 1, (255, 0, 0), 2
            )
            cv.imshow(window, img)

    cv.namedWindow(window, cv.WINDOW_NORMAL)
    cv.imshow(window, img)
    cv.setMouseCallback(window, click_event, [box_corners, img, window])
    # wait for a key to be pressed to exit
    # print("click on the 4 corners, then press any key.")
    cv.waitKey(0)

    tl, bl, tr, br = np.array(box_corners, dtype=np.float32)

    # get first and last columns of points with linear interpolation
    first_col = np.linspace(tl, bl, n_rows)
    last_col = np.linspace(tr, br, n_rows)

    # get rest corner points by linearly interpolating the two columns
    all_points = np.vstack(
        [np.linspace(first_col[i], last_col[i], n_cols) for i in range(n_rows)]
    )
    corners = all_points.reshape(-1, 1, 2)

    objpoints.append(objp)
    imgpoints.append(corners)

    cv.destroyWindow(window)
    cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
    cv.imshow("Interpolated Chessboard Corners", img)
    cv.waitKey(0)
    cv.destroyAllWindows()


def find_auto(img, gray, objpoints, imgpoints, objp, window, n_rows=9, n_cols=6):
    """
    used to find the corner points and fill objpoints, imgpoints lists
    img, gray: original and grayscale image
    """

    # choice task - denoising
    gray = cv.fastNlMeansDenoising(gray, None, h=10)

    # detect chess board corners
    ret, corners = cv.findChessboardCorners(gray, (n_rows, n_cols), None)

    if ret:
        # improve quality of automatically found corners
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners)

        # display the corners on image
        cv.drawChessboardCorners(img, (n_rows, n_cols), corners, True)
        cv.imshow(window, img)
        cv.waitKey(0)
        cv.destroyWindow(window)
    else:
        # if corners are not found, switch to manual mode
        click_corners(img, objpoints, imgpoints, objp, window, n_rows, n_cols)


def flat_str(matrix, vec=False):

    if vec:
        flat = [str(element) for element in matrix.flatten().tolist()]
    else:
        flat = ["  ".join(str(element) for element in row) for row in matrix.tolist()]
    return "\n  " + "\n  ".join(flat) + "\n"


def save_config(mtx, dist, R, tvec, video=1):

    opencv_storage = ET.Element("opencv_storage")
    camera_matrix = ET.SubElement(
        opencv_storage, "CameraMatrix", type_id="opencv-matrix"
    )
    ET.SubElement(camera_matrix, "rows").text = "3"
    ET.SubElement(camera_matrix, "cols").text = "3"
    ET.SubElement(camera_matrix, "dt").text = "f"
    ET.SubElement(camera_matrix, "data").text = flat_str(mtx)

    distortion_coeffs_elem = ET.SubElement(
        opencv_storage, "DistortionCoeffs", type_id="opencv-matrix"
    )
    ET.SubElement(distortion_coeffs_elem, "rows").text = "5"
    ET.SubElement(distortion_coeffs_elem, "cols").text = "1"
    ET.SubElement(distortion_coeffs_elem, "dt").text = "f"
    ET.SubElement(distortion_coeffs_elem, "data").text = flat_str(dist, True)

    rotation_matrix_elem = ET.SubElement(
        opencv_storage, "RotationMatrix", type_id="opencv-matrix"
    )
    ET.SubElement(rotation_matrix_elem, "rows").text = "3"
    ET.SubElement(rotation_matrix_elem, "cols").text = "3"
    ET.SubElement(rotation_matrix_elem, "dt").text = "f"
    ET.SubElement(rotation_matrix_elem, "data").text = flat_str(R)

    translation_vector_elem = ET.SubElement(
        opencv_storage, "TranslationVector", type_id="opencv-matrix"
    )
    ET.SubElement(translation_vector_elem, "rows").text = "3"
    ET.SubElement(translation_vector_elem, "cols").text = "1"
    ET.SubElement(translation_vector_elem, "dt").text = "f"
    ET.SubElement(translation_vector_elem, "data").text = flat_str(tvec, True)

    camera = "cam" + str(video)
    tree = ET.ElementTree(opencv_storage)
    # write to file
    with open(f"data/{camera}/config.xml", "wb") as file:
        tree.write(file, encoding="utf-8", xml_declaration=True)


def get_checkboard(video=1):

    camera = "cam" + str(video)
    path = f"./data/{camera}/checkerboard.avi"
    cam = cv.VideoCapture(path)

    if not cam.isOpened():
        print("Error: Could not open video file.")

    j = 0
    while True:

        ret, frame = cam.read()
        if ret:
            j += 1
            if j == 1:
                out_path = f"data/{camera}/checkboard_axes.jpg"
                cv.imwrite(out_path, frame)
                print(f"Saved {out_path}")
            if j >= 5:
                break
        else:
            print("failed to grab frame")
            break
    cam.release()

## Intrinsics


In [3]:
def calibrate(video=1, n_rows=8, n_cols=6):

    vid = {
        1: [1, 30, 100, 120, 160, 200],
        2: [30, 60, 160, 400, 700, 800],
        3: [230, 260, 280, 530, 800],
        4: [1, 30, 60, 160, 180, 200, 260, 300],
    }

    objp = np.zeros((n_rows * n_cols, 3), np.float32)

    objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)

    objp = objp * 115

    # Lists to store object and image points from all images.

    objpoints = []  # 3d point in real world space

    imgpoints = []  # 2d points in image plane.

    for fr in vid[video]:

        frame = f"cam{video}_image{fr}.jpg"

        img = cv.imread(frame)

        if img is None:

            continue

        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # we convert to grayscale before passing to 'findChessboardCorners'
        window = frame
        # find chess board corners either automatically or manually
        find_auto(
            img, gray, objpoints, imgpoints, objp, window, n_rows=n_rows, n_cols=n_cols
        )

    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None
    )

    return ret, mtx, dist, rvecs, tvecs, objpoints, imgpoints

## Extrinsics


In [ ]:
def camera_params(video=1, n_rows=8, n_cols=6):

    ret, mtx, dist, rvecs, tvecs, objpoints, imgpoints = calibrate(video)

    # for extrinsics use checkerboard.avi
    get_checkboard(video)

    objp = np.zeros((n_rows * n_cols, 3), np.float32)
    objp[:, :2] = np.mgrid[0:n_rows, 0:n_cols].T.reshape(-1, 2)
    objp = objp * 115
    objpoints = []
    imgpoints = []

    camera = "cam" + str(video)
    img = cv.imread(f"data/{camera}/checkboard_axes.jpg")
    window = f"extrinsic_{camera}"
    copy = img.copy()

    click_corners(
        img,
        objpoints,
        imgpoints,
        objp,
        window=window,
        n_rows=n_rows,
        n_cols=n_cols,
    )

    success, rvec, tvec = cv.solvePnP(
        objpoints[0], imgpoints[0], mtx, dist, useExtrinsicGuess=False
    )

    if success:
        R, _ = cv.Rodrigues(rvec)

        save_config(mtx, dist, R, tvec, video)

        drawn_img = drawAxes(copy, rvec, tvec, mtx, dist)
        cv.imshow("3D Axes on Checkerboard", drawn_img)
        cv.waitKey(0)
        cv.destroyAllWindows()


def drawAxes(img, rvec, tvec, mtx, dist):

    square = 115
    # coordinate system
    axis = np.float32(
        [[0, 0, 0], [3 * square, 0, 0], [0, 3 * square, 0], [0, 0, -3 * square]]
    )

    # project the axis points and cube points to the 2D image and then convert to pixel coordinates
    imgpts_axis, _ = cv.projectPoints(axis, rvec, tvec, mtx, dist)

    origin = tuple(map(int, imgpts_axis[0].ravel()))
    pt_x = tuple(map(int, imgpts_axis[1].ravel()))
    pt_y = tuple(map(int, imgpts_axis[2].ravel()))
    pt_z = tuple(map(int, imgpts_axis[3].ravel()))
    cv.circle(img, origin, 5, (0, 255, 255), -1)

    cv.arrowedLine(img, origin, pt_x, (0, 0, 255), 4, tipLength=0.2)  # X
    cv.arrowedLine(img, origin, pt_y, (0, 255, 0), 4, tipLength=0.2)  # Y
    cv.arrowedLine(img, origin, pt_z, (255, 0, 0), 4, tipLength=0.2)  # Z
    cv.putText(img, "X", pt_x, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, "Y", pt_y, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)
    cv.putText(img, "Z", pt_z, cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv.LINE_AA)

    return img

In [5]:
camera_params(1)
camera_params(2)
camera_params(3)
camera_params(4)

Saved data/cam1/checkboard_axes.jpg
Saved data/cam2/checkboard_axes.jpg
Saved data/cam3/checkboard_axes.jpg
Saved data/cam4/checkboard_axes.jpg


In [6]:
# path = "./data/cam2/intrinsics.avi"
# cam = cv.VideoCapture(path)

# if not cam.isOpened():
#     print("Error: Could not open video file.")


# j = 0
# while True:

#     ret, frame = cam.read()
#     if ret:
#         j += 1

#         if j in [300, 400, 500, 700, 800]:
#             # [1, 30, 60, 80, 100, 120, 160, 180, 200, 230, 260, 300]
#             out_path = f"cam2_image{j}.jpg"
#             cv.imwrite(out_path, frame)
#             print(f"Saved {out_path}")

#         if j >= 1000:
#             break
#     else:
#         print("failed to grab frame")
#         break

# cam.release()